In [3]:
import json
from datetime import datetime

from mysql.connector import Error
import mysql.connector as msql
from pymongo import MongoClient

# filename = 'corona-out-2'
filename = 'corona-out-3'

In [4]:
conn = msql.connect(host='localhost', user='root', password='Password')
cursor = conn.cursor()

In [5]:
mconn = MongoClient()

In [6]:
def create_mysql_db():
            cursor.execute("CREATE DATABASE TwitterDB;")
            cursor.execute("USE TwitterDB;")
            cursor.execute("CREATE TABLE users_data(id varchar(255),name varchar(255),screen_name varchar(255),location varchar(255),url varchar(255),description varchar(255),verified TINYINT,followers_count int,friends_count int,listed_count int,favourites_count int,statuses_count int,created_at int)")


def add_user_to_mysql(user):
            sql = "INSERT INTO TwitterDB.users_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(user.values()))
            conn.commit()


def check_user_in_mysql(id):
    query = f"SELECT count(*) FROM users_data WHERE id = '{id}';"
    cursor.execute(query)
    a=cursor.fetchall()
    return a
            

def add_tweet_to_mongo(tweet):


    db = mconn.TwitterDB
    collection = db.Tweets
    collection.insert_one(tweet)
    
def add_retweet_to_mongo(retweet):


    db = mconn.TwitterDB
    collection = db.Retweets
    collection.insert_one(retweet)

In [7]:
create_mysql_db()

In [8]:
def to_timestamp(val):
    dt = val.split(' ')
    st = f"{dt[5]}-{dt[1]}-{dt[2]} {dt[3]}"
    dt = datetime.strptime(st, "%Y-%b-%d %H:%M:%S")
    return int(dt.timestamp())

def to_dtstring(val):
    return datetime.fromtimestamp(val).strftime('%b %d, %Y - %X')

def get_hashtags(data):
    hashtags = []
    for item in data:
        hashtags.append(item['text'])
    return hashtags

In [9]:


def add_user(user):
    if (user['id_str'] not in users):
        new_user = {}
        
        new_user['id'] = user['id_str']
        new_user['name'] = user['name']
        new_user['screen_name'] = user['screen_name']
        new_user['location'] = user['location']
        new_user['url'] = user['url']
        new_user['description'] = user['description']
        new_user['verified'] = user['verified']
        new_user['followers_count'] = user['followers_count']
        new_user['friends_count'] = user['friends_count']
        new_user['listed_count'] = user['listed_count']
        new_user['favourites_count'] = user['favourites_count']
        new_user['statuses_count'] = user['statuses_count']
        new_user['created_at'] = to_timestamp(user['created_at'])
        
        users[user['id_str']] = new_user
        add_user_to_mysql(new_user)


def process_tweet(tweet):
    new_tweet = {}
        
    new_tweet['created_at'] = to_timestamp(tweet['created_at'])
    new_tweet['id'] = tweet['id_str']
    if tweet['truncated']:
        new_tweet['text'] = tweet['extended_tweet']['full_text']
        new_tweet['hashtags'] = get_hashtags(tweet['extended_tweet']['entities']['hashtags'])
    else:
        new_tweet['text'] = tweet['text']
        new_tweet['hashtags'] = get_hashtags(tweet['entities']['hashtags'])

    user = {}
    user['id'] = tweet['user']['id_str']
    user['name'] = tweet['user']['name']
    user['screen_name'] = tweet['user']['screen_name']
    user['verified'] = tweet['user']['verified']
    # updated
    user['friends_count'] = tweet['user']['friends_count']
    user['followers_count'] = tweet['user']['followers_count']
    user['description'] = tweet['user']['description']
    user['created_at'] = tweet['user']['created_at']
    new_tweet['user'] = user

    new_tweet['source'] = tweet['source']
    new_tweet['quote_count'] = tweet['quote_count']
    new_tweet['reply_count'] = tweet['reply_count']
    new_tweet['retweet_count'] = tweet['retweet_count']
    new_tweet['favorite_count'] = tweet['favorite_count']
    
    new_tweet['score'] = 3*tweet['retweet_count']
    new_tweet['score'] += 3*tweet['reply_count']
    new_tweet['score'] += 2*tweet['quote_count']
    new_tweet['score'] += 2*tweet['favorite_count']
    if tweet['user']['verified']:
        new_tweet['score'] *= 2
    
    if 'retweeted_status' in tweet:
        new_tweet['retweeted_status'] = process_tweet(tweet['retweeted_status'])
        
    new_tweet['is_quote_status'] = tweet['is_quote_status']
    if tweet['is_quote_status'] and 'quoted_status' in tweet:
        new_tweet['quoted_status'] = process_tweet(tweet['quoted_status'])
    
    return new_tweet
        
def add_tweet(tweet, tweet_type):
    if (tweet['id_str'] not in tweets):
        
        new_tweet = process_tweet(tweet)
        
        if tweet_type=='tweet':
            add_tweet_to_mongo(new_tweet)
            tweets[tweet['id_str']] = new_tweet
        elif tweet_type=='retweet':
            add_retweet_to_mongo(new_tweet)
            retweets[tweet['id_str']] = new_tweet
#         elif tweet_type=='quote':
#             quotes[tweet['id_str']] = new_tweet

In [10]:
def process_data(data):
    if data['id_str'] in tweets:
        return
    
    add_user(data['user'])

    if data['text'].startswith('RT'):
        add_tweet(data, 'retweet')
        if 'retweeted_status' in data:
            process_data(data['retweeted_status'])

    else:
        add_tweet(data, 'tweet')

In [11]:
users = {}
tweets = {}
retweets = {}
quotes = {}

for line in open(filename, 'r'):
    if len(line.strip()) == 0:
        continue
        
#     try:
    data = json.loads(line)
    process_data(data)

#     except:
#         continue

In [12]:
# (7349, 11157, 15505)
# (10987, 11157, 18461)

len(tweets), len(retweets), len(users), len(quotes)

(50915, 61108, 88068, 0)

In [13]:
mconn.close()
cursor.close()
conn.close()